In [2]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.<INSERT_VERSION>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-03-16 12:35:06--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.30MB/s    in 0.7s    

2021-03-16 12:35:07 (1.30 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Zillow Housing Prices").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
# Read in data from S3 Bucket
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/unit-7/zillow_housing_prices.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("zillow_housing_prices.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+---+--------+-------------+--------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+--------

In [7]:
df = df.withColumn("2020-01", df["2020-01"].cast('float'))
df = df.withColumn("2020-02", df["2020-02"].cast('float'))
df = df.withColumn("2020-03", df["2020-03"].cast('float'))

In [8]:
stateGrouped = df.groupBy('StateName')
stateGrouped = stateGrouped.agg({'2020-01':'Avg'})

In [9]:
stateGrouped.orderBy(stateGrouped['avg(2020-01)'].desc()).show()

+--------------------+------------------+
|           StateName|      avg(2020-01)|
+--------------------+------------------+
|          California| 616401.3192612138|
|District of Columbia|          580100.0|
|              Hawaii|         572868.75|
|          Washington|438228.30188679247|
|       Massachusetts| 427036.0294117647|
|            Colorado|407714.81481481483|
|            New York|          388424.8|
|          New Jersey| 368433.1491712707|
|             Montana|357857.14285714284|
|            Virginia| 345496.8085106383|
|                Utah|          345200.0|
|              Oregon| 342466.6666666667|
|            Maryland| 328265.2173913043|
|        Rhode Island|          327400.0|
|              Nevada|          327370.0|
|       New Hampshire| 308258.3333333333|
|             Arizona| 301616.9811320755|
|         Connecticut|291745.28301886795|
|            Delaware| 276073.6842105263|
|               Idaho|271421.73913043475|
+--------------------+------------

In [10]:
februaryGroup = df.groupby('StateName')
februaryGroup = februaryGroup.agg({'2020-02':'Avg'})

In [11]:
februaryGroup.orderBy(februaryGroup['avg(2020-02)'].desc()).show()

+--------------------+------------------+
|           StateName|      avg(2020-02)|
+--------------------+------------------+
|          California| 628730.8108108108|
|              Hawaii|          592525.0|
|District of Columbia|          575700.0|
|          Washington| 439433.3333333333|
|       Massachusetts| 428246.6165413534|
|            Colorado|407444.44444444444|
|            New York| 393853.2786885246|
|          New Jersey| 371965.9090909091|
|             Montana|368942.85714285716|
|            Virginia|350971.27659574465|
|              Oregon|348658.82352941175|
|                Utah|  347562.962962963|
|            Maryland| 334171.6814159292|
|        Rhode Island| 329465.2173913043|
|              Nevada|          327770.0|
|       New Hampshire| 316154.1666666667|
|             Arizona|308584.90566037735|
|         Connecticut| 297518.8679245283|
|            Delaware| 285189.4736842105|
|               Idaho|275943.47826086957|
+--------------------+------------

In [12]:
februaryGroup.filter('avg(2020-02)>2000').show()

+--------------------+------------------+
|           StateName|      avg(2020-02)|
+--------------------+------------------+
|                Utah|  347562.962962963|
|              Hawaii|          592525.0|
|           Minnesota|261769.76744186046|
|                Ohio|182818.30065359478|
|              Oregon|348658.82352941175|
|            Arkansas|165128.57142857142|
|               Texas| 269829.6703296703|
|        North Dakota|          232660.0|
|        Pennsylvania| 250119.6261682243|
|         Connecticut| 297518.8679245283|
|            Nebraska|          198640.0|
|              Nevada|          327770.0|
|          Washington| 439433.3333333333|
|            Illinois|232948.19277108434|
|            Oklahoma|165754.83870967742|
|District of Columbia|          575700.0|
|            Delaware| 285189.4736842105|
|          New Mexico|          228962.5|
|       West Virginia|          184400.0|
|            Missouri|209020.83333333334|
+--------------------+------------

In [13]:
output = df['RegionID', 'RegionName','StateName','2020-01','2020-02','2020-03']

In [14]:
output.show()

+--------+-------------+--------------+---------+---------+--------+
|RegionID|   RegionName|     StateName|  2020-01|  2020-02| 2020-03|
+--------+-------------+--------------+---------+---------+--------+
|    6181|     New York|      New York| 571700.0| 568300.0|573600.0|
|   12447|  Los Angeles|    California| 738200.0| 760200.0|    null|
|   39051|      Houston|         Texas| 219200.0| 223800.0|    null|
|   17426|      Chicago|      Illinois| 281400.0| 302900.0|309200.0|
|    6915|  San Antonio|         Texas| 205400.0| 208300.0|    null|
|   13271| Philadelphia|  Pennsylvania| 184100.0| 188300.0|    null|
|   40326|      Phoenix|       Arizona| 250600.0| 254700.0|    null|
|   18959|    Las Vegas|        Nevada| 278900.0| 283000.0|284700.0|
|   54296|    San Diego|    California| 617000.0| 619200.0|    null|
|   38128|       Dallas|         Texas| 321500.0| 315500.0|321700.0|
|   10221|       Austin|         Texas| 399900.0| 397800.0|    null|
|   33839|     San Jose|    Califo

In [ ]:
postgres_url="jdbc:postgresql://<INSERT RDS URL>:5432/<INSERT DB NAME>"
config = {"user":"postgres", 
          "password": "<PASSWORD>", 
          "driver":"org.postgresql.Driver"}
output.write.jdbc(postgres_url,table="zillowhousingprices",mode="append", properties=config)